# 图片放大

In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
from pathlib import Path
import cv2, shutil


TARGET_MIN = 640      # 短边放大到这个尺寸（可改 320/384/512）

def upscale(img):
    h, w = img.shape[:2]
    if min(h, w) >= TARGET_MIN:
        return img
    s = TARGET_MIN / min(h, w)
    nh, nw = int(round(h*s)), int(round(w*s))
    # Lanczos 放大
    up = cv2.resize(img, (nw, nh), interpolation=cv2.INTER_LANCZOS4)
    # 轻微锐化（unsharp mask）
    blur = cv2.GaussianBlur(up, (0,0), 1.0)
    sharp = cv2.addWeighted(up, 1.15, blur, -0.15, 0)
    return sharp

def process_split(split, SRC, DST):
    for p in (SRC/split).rglob("*"):
        if p.suffix.lower() not in {".jpg",".jpeg",".png"}: 
            continue
        rel = p.relative_to(SRC)
        img = cv2.imread(str(p))
        if img is None: 
            continue
        out_img = upscale(img)
        out_path = (DST/rel).with_suffix(".jpg")
        out_path.parent.mkdir(parents=True, exist_ok=True)
        cv2.imwrite(str(out_path), out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 95])


In [7]:
DEST_DIR = Path(f"/workspace/data/cropped_objects_refined_data_v2_v3")
for split in ["swd","mayswd","others"]:
    process_split(split, DEST_DIR, DEST_DIR)
    print("✅ Upscale done.")

✅ Upscale done.
✅ Upscale done.
✅ Upscale done.


# 同名json文件和image。 同步H,W大小

In [3]:
import json
import os
from PIL import Image
import glob

def update_image_dimensions_in_json(json_file_path):
    """
    读取JSON文件，获取imagePath对应的图片，更新imageHeight和imageWidth
    """
    try:
        # 读取JSON文件
        with open(json_file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        # 获取图片路径
        image_path = data.get('imagePath', '')
        if not image_path:
            print(f"Warning: No imagePath found in {json_file_path}")
            return False
        
        # 构建完整的图片路径（假设图片在同一目录或相对路径）
        image_full_path = os.path.join(os.path.dirname(json_file_path), image_path)
        
        if not os.path.exists(image_full_path):
            print(f"Warning: Image file not found: {image_full_path}")
            return False
        
        # 使用PIL读取图片尺寸
        with Image.open(image_full_path) as img:
            width, height = img.size
        
        # 更新JSON数据
        data['imageHeight'] = height
        data['imageWidth'] = width
        
        # 写回JSON文件
        with open(json_file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
        
        print(f"Updated {json_file_path}: {width}x{height}")
        return True
        
    except Exception as e:
        print(f"Error processing {json_file_path}: {e}")
        return False

def batch_update_json_dimensions(json_dir):
    """
    批量处理目录中的所有JSON文件
    """
    # 查找所有的JSON文件
    json_files = glob.glob(os.path.join(json_dir, "**", "*.json"), recursive=True)
    
    success_count = 0
    total_count = len(json_files)
    
    for json_file in json_files:
        if update_image_dimensions_in_json(json_file):
            success_count += 1
    
    print(f"Processed {success_count}/{total_count} JSON files successfully")

# 使用示例
if __name__ == "__main__":
    # 处理单个文件
    # json_file_path = "/path/to/your/file.json"
    # update_image_dimensions_in_json(json_file_path)
    
    # 批量处理目录
    json_directory = "/workspace/models/runs_yolov11_pose/cropped_objects/mayswd"  # 替换为你的JSON文件目录
    batch_update_json_dimensions(json_directory)

Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0604_580_obj14_swd_uuid_d1d94c90-a467-47d0-b016-e6ebfa103e8d.json: 640x640
Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0607_640_obj35_swd_uuid_966802cc-f544-4fae-a1d4-0db8ea6d2f51.json: 640x640
Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0634_580_obj94_swd_uuid_840d5d0c-7ef4-423a-b118-27bb28cc9f62.json: 640x640
Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0634_580_obj95_swd_uuid_05727c5d-7df8-45ae-a2be-e3e13d2c41c1.json: 647x640
Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0704_580_obj105_swd_uuid_fa3e1123-d08f-4c11-8f32-a4ba82fe921c.json: 640x649
Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0704_580_obj33_swd_uuid_484fa83e-0b1a-4c1a-887d-e4faf6ba60e1.json: 640x640
Updated /workspace/models/runs_yolov11_pose/cropped_objects/mayswd/0718_0704_580_obj57_swd_uuid_30bc683b-1c23-480b-ad